### This notebook shows the extraction of tweets, cleaning of tweets and usage of VADER sentiment analysis to generate the sentiment score for each tweet which can be futher used in the prediction of Dogecoin prices. 

# Import Libraries 

In [12]:
import re # regular expressions
from tqdm import tnrange, tqdm_notebook, tqdm
import nltk
import warnings
warnings.filterwarnings('ignore')
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
import pandas as pd
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import dateutil.parser as dparser
import spacy

from nltk import tokenize
from datetime import datetime, timedelta
import time
import pprint
from nltk import word_tokenize
from statistics import mean
import emoji 
import pandas as pd
import re
import emoji
import string 
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())

from Scweet.scweet import scrap
import pandas as pd


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Ruchil\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


# Data Preprocessing 

In [8]:
# Load dataset
def load_data():
    data = pd.read_csv('C:/Users/Ruchil/OneDrive/MS_Analytics_HU/5thSemester/dataproject/Tweets_Dogecoin.csv')
    return data

In [9]:
tweet_df = load_data()
tweet_df.head()

,Unnamed: 0,Timestamp,Text
0,0,2021-01-01T23:06:39.000Z,Dogecoin to the moon!? #dogecoin
1,1,2021-01-01T22:39:16.000Z,What a great way to start the new year with #D...
2,2,2021-01-01T23:43:32.000Z,Imagine waking up one day and finding out #Dog...
3,3,2021-01-01T23:38:15.000Z,#dogecoin now at 15% up today. can we do 20?
4,4,2021-01-01T21:38:15.000Z,If you haven’t invested in DOGECOIN yet I real...


In [5]:
print('Dataset size:',tweet_df.shape)
print('Columns are:',tweet_df.columns)

Dataset size: (8207, 3)
Columns are: Index(['Unnamed: 0', 'Timestamp', 'Text'], dtype='object')


In [6]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8207 entries, 0 to 8206
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  8207 non-null   int64 
 1   Timestamp   8207 non-null   object
 2   Text        8207 non-null   object
dtypes: int64(1), object(2)
memory usage: 192.5+ KB


In [10]:
df  = pd.DataFrame(tweet_df[['Timestamp', 'Text']])

# Data Cleaning 

### Remove punctuations

In [13]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [14]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('â€™', '', text)
    
    return text
df['Tweet_punct'] = df['Text'].map(remove_punct)
df.to_csv(r'C:\Users\Ruchil\OneDrive\MS_Analytics_HU\5thSemester\dataproject\Cleaned_Tweets.csv')
df.head(10)

,Timestamp,Text,Tweet_punct
0,2021-01-01T23:06:39.000Z,Dogecoin to the moon!? #dogecoin,Dogecoin to the moon dogecoin
1,2021-01-01T22:39:16.000Z,What a great way to start the new year with #D...,What a great way to start the new year with Do...
2,2021-01-01T23:43:32.000Z,Imagine waking up one day and finding out #Dog...,Imagine waking up one day and finding out Doge...
3,2021-01-01T23:38:15.000Z,#dogecoin now at 15% up today. can we do 20?,dogecoin now at up today can we do
4,2021-01-01T21:38:15.000Z,If you haven’t invested in DOGECOIN yet I real...,If you haven’t invested in DOGECOIN yet I real...
5,2021-01-01T22:11:36.000Z,buy and hold dogecoin should be 3 in marketcap...,buy and hold dogecoin should be in marketcap ...
6,2021-01-01T22:28:22.000Z,$Doge isn’t stopping! #Dogecoin #Doge #DogeArm...,Doge isn’t stopping Dogecoin Doge DogeArmy cry...
7,2021-01-01T20:21:05.000Z,#DogeCoin #CryptoCurrency #On #The #Move #Chat...,DogeCoin CryptoCurrency On The Move Chat Doge ...
8,2021-01-01T19:29:39.000Z,Where the money resides #dogecoin,Where the money resides dogecoin
9,2021-01-01T23:33:19.000Z,I’m going to buy a Tesla w/my $Doge money and ...,I’m going to buy a Tesla wmy Doge money and th...


### Tokenization 

In [15]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

df['Tweet_tokenized'] = df['Tweet_punct'].apply(lambda x: tokenization(x.lower()))
df.head()

,Timestamp,Text,Tweet_punct,Tweet_tokenized
0,2021-01-01T23:06:39.000Z,Dogecoin to the moon!? #dogecoin,Dogecoin to the moon dogecoin,"[dogecoin, to, the, moon, dogecoin]"
1,2021-01-01T22:39:16.000Z,What a great way to start the new year with #D...,What a great way to start the new year with Do...,"[what, a, great, way, to, start, the, new, yea..."
2,2021-01-01T23:43:32.000Z,Imagine waking up one day and finding out #Dog...,Imagine waking up one day and finding out Doge...,"[imagine, waking, up, one, day, and, finding, ..."
3,2021-01-01T23:38:15.000Z,#dogecoin now at 15% up today. can we do 20?,dogecoin now at up today can we do,"[dogecoin, now, at, up, today, can, we, do, ]"
4,2021-01-01T21:38:15.000Z,If you haven’t invested in DOGECOIN yet I real...,If you haven’t invested in DOGECOIN yet I real...,"[if, you, haven, t, invested, in, dogecoin, ye..."


### Remove stopwords

In [16]:
stopword = nltk.corpus.stopwords.words('english')

def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stopword])
df['Tweet_nonstop'] = df['Tweet_punct'].apply(lambda text: cleaning_stopwords(text))
df['Tweet_nonstop'] = df['Tweet_nonstop'].str.lower()
df['Tweet_nonstop'].head()


### Stemming and Lammitization

In [17]:
# Remove Stop words 
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
df['Tweet_nonstopwords'] = df['Tweet_tokenized'].apply(lambda x: remove_stopwords(x))


ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

df['Tweet_stemmed'] = df['Tweet_nonstopwords'].apply(lambda x: stemming(x))
df.head()

,Timestamp,Text,Tweet_punct,Tweet_tokenized,Tweet_nonstop,Tweet_nonstopwords,Tweet_stemmed
0,2021-01-01T23:06:39.000Z,Dogecoin to the moon!? #dogecoin,Dogecoin to the moon dogecoin,"[dogecoin, to, the, moon, dogecoin]",dogecoin moon dogecoin,"[dogecoin, moon, dogecoin]","[dogecoin, moon, dogecoin]"
1,2021-01-01T22:39:16.000Z,What a great way to start the new year with #D...,What a great way to start the new year with Do...,"[what, a, great, way, to, start, the, new, yea...",what great way start new year dogecoin going,"[great, way, start, new, year, dogecoin, going, ]","[great, way, start, new, year, dogecoin, go, ]"
2,2021-01-01T23:43:32.000Z,Imagine waking up one day and finding out #Dog...,Imagine waking up one day and finding out Doge...,"[imagine, waking, up, one, day, and, finding, ...",imagine waking one day finding dogecoin shot s...,"[imagine, waking, one, day, finding, dogecoin,...","[imagin, wake, one, day, find, dogecoin, shot,..."
3,2021-01-01T23:38:15.000Z,#dogecoin now at 15% up today. can we do 20?,dogecoin now at up today can we do,"[dogecoin, now, at, up, today, can, we, do, ]",dogecoin today,"[dogecoin, today, ]","[dogecoin, today, ]"
4,2021-01-01T21:38:15.000Z,If you haven’t invested in DOGECOIN yet I real...,If you haven’t invested in DOGECOIN yet I real...,"[if, you, haven, t, invested, in, dogecoin, ye...",if haven’t invested dogecoin yet i really can’...,"[invested, dogecoin, yet, really, anything, else]","[invest, dogecoin, yet, realli, anyth, els]"


In [18]:

wn = nltk.WordNetLemmatizer()


def lemmatizer(text):
    return " ".join([wn.lemmatize(word) for word in str(text).split()])
df['Tweet_lemmatized'] = df['Tweet_nonstop'].apply(lambda x: lemmatizer(x))
df.head()

,Timestamp,Text,Tweet_punct,Tweet_tokenized,Tweet_nonstop,Tweet_nonstopwords,Tweet_stemmed,Tweet_lemmatized
0,2021-01-01T23:06:39.000Z,Dogecoin to the moon!? #dogecoin,Dogecoin to the moon dogecoin,"[dogecoin, to, the, moon, dogecoin]",dogecoin moon dogecoin,"[dogecoin, moon, dogecoin]","[dogecoin, moon, dogecoin]",dogecoin moon dogecoin
1,2021-01-01T22:39:16.000Z,What a great way to start the new year with #D...,What a great way to start the new year with Do...,"[what, a, great, way, to, start, the, new, yea...",what great way start new year dogecoin going,"[great, way, start, new, year, dogecoin, going, ]","[great, way, start, new, year, dogecoin, go, ]",what great way start new year dogecoin going
2,2021-01-01T23:43:32.000Z,Imagine waking up one day and finding out #Dog...,Imagine waking up one day and finding out Doge...,"[imagine, waking, up, one, day, and, finding, ...",imagine waking one day finding dogecoin shot s...,"[imagine, waking, one, day, finding, dogecoin,...","[imagin, wake, one, day, find, dogecoin, shot,...",imagine waking one day finding dogecoin shot s...
3,2021-01-01T23:38:15.000Z,#dogecoin now at 15% up today. can we do 20?,dogecoin now at up today can we do,"[dogecoin, now, at, up, today, can, we, do, ]",dogecoin today,"[dogecoin, today, ]","[dogecoin, today, ]",dogecoin today
4,2021-01-01T21:38:15.000Z,If you haven’t invested in DOGECOIN yet I real...,If you haven’t invested in DOGECOIN yet I real...,"[if, you, haven, t, invested, in, dogecoin, ye...",if haven’t invested dogecoin yet i really can’...,"[invested, dogecoin, yet, really, anything, else]","[invest, dogecoin, yet, realli, anyth, els]",if haven’t invested dogecoin yet i really can’...


### Converting date 

In [19]:
df['Date'] = pd.to_datetime(df['Timestamp'], errors='coerce')
df = df.dropna(axis=0) 
df[pd.isnull(pd.to_datetime(df.Date))]
df.head()

,Timestamp,Text,Tweet_punct,Tweet_tokenized,Tweet_nonstop,Tweet_nonstopwords,Tweet_stemmed,Tweet_lemmatized,Date
0,2021-01-01T23:06:39.000Z,Dogecoin to the moon!? #dogecoin,Dogecoin to the moon dogecoin,"[dogecoin, to, the, moon, dogecoin]",dogecoin moon dogecoin,"[dogecoin, moon, dogecoin]","[dogecoin, moon, dogecoin]",dogecoin moon dogecoin,2021-01-01 23:06:39+00:00
1,2021-01-01T22:39:16.000Z,What a great way to start the new year with #D...,What a great way to start the new year with Do...,"[what, a, great, way, to, start, the, new, yea...",what great way start new year dogecoin going,"[great, way, start, new, year, dogecoin, going, ]","[great, way, start, new, year, dogecoin, go, ]",what great way start new year dogecoin going,2021-01-01 22:39:16+00:00
2,2021-01-01T23:43:32.000Z,Imagine waking up one day and finding out #Dog...,Imagine waking up one day and finding out Doge...,"[imagine, waking, up, one, day, and, finding, ...",imagine waking one day finding dogecoin shot s...,"[imagine, waking, one, day, finding, dogecoin,...","[imagin, wake, one, day, find, dogecoin, shot,...",imagine waking one day finding dogecoin shot s...,2021-01-01 23:43:32+00:00
3,2021-01-01T23:38:15.000Z,#dogecoin now at 15% up today. can we do 20?,dogecoin now at up today can we do,"[dogecoin, now, at, up, today, can, we, do, ]",dogecoin today,"[dogecoin, today, ]","[dogecoin, today, ]",dogecoin today,2021-01-01 23:38:15+00:00
4,2021-01-01T21:38:15.000Z,If you haven’t invested in DOGECOIN yet I real...,If you haven’t invested in DOGECOIN yet I real...,"[if, you, haven, t, invested, in, dogecoin, ye...",if haven’t invested dogecoin yet i really can’...,"[invested, dogecoin, yet, really, anything, else]","[invest, dogecoin, yet, realli, anyth, els]",if haven’t invested dogecoin yet i really can’...,2021-01-01 21:38:15+00:00


# VADER Sentiment Analysis (Generating sentiment score)

In [24]:
tweets = df 
tweetsWithSent = []

for t in tweets['Tweet_lemmatized']:
    
    ps = analyzer.polarity_scores(t)
    tweetsWithSent.append({'Tweet_lemmatized':t, 'compound':ps['compound']})

In [25]:
import pandas as pd
tweet_df = pd.DataFrame(tweetsWithSent)


In [26]:

tweet_df.rename(columns={"Tweet_lemmatized": "tweets", "compound": "sentiment_score"}, inplace = True)
tweet_df
# The dataframe shows the tweets along with their sentiment score

,tweets,sentiment_score
0,dogecoin moon dogecoin,0.0000
1,what great way start new year dogecoin going,0.6249
2,imagine waking one day finding dogecoin shot s...,0.5719
3,dogecoin today,0.0000
4,if haven’t invested dogecoin yet i really can’...,0.0000
...,...,...
8202,people speculating motive small pump well opin...,0.5719
8203,what dogecoin the abc billion joke httpstribal...,0.6597
8204,my two favorite coin market kishu banksocial l...,0.2263
8205,the current price one doge change last hourone...,0.2263


In [ ]:
tweet_df.plot.bar(figsize=(15,5),width=1)
# The below mentioned plot shows the sentiment score of each tweet, 
#by this plot we can understand most of the tweets are positive 

In [ ]:
# These sentiment stores will be used to predict the dogecoin price with the usage of LSTM-RNN. 

Referances :
https://towardsdatascience.com/sentiment-analysis-of-tweets-167d040f0583
https://www.kaggle.com/ragnisah/text-data-cleaning-tweets-analysis

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import tweepy
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [2]:
analyser.polarity_scores("I am getting bored in the quarantine.")

{'neg': 0.259, 'neu': 0.741, 'pos': 0.0, 'compound': -0.2732}

In [22]:
analyzer = SentimentIntensityAnalyzer()
scores = []
# Declare variables for scores
compound_list = []
positive_list = []
negative_list = []
neutral_list = []
for i in range(df['Tweet_lemmatized'].shape[0]):
#print(analyser.polarity_scores(sentiments_pd['text'][i]))
    compound = analyzer.polarity_scores(df['Tweet_lemmatized'][i])["compound"]
    pos = analyzer.polarity_scores(df['Tweet_lemmatized'][i])["pos"]
    neu = analyzer.polarity_scores(df['Tweet_lemmatized'][i])["neu"]
    neg = analyzer.polarity_scores(df['Tweet_lemmatized'][i])["neg"]
    
    scores.append({"Compound": compound,
                       "Positive": pos,
                       "Negative": neg,
                       "Neutral": neu
                  })

In [39]:
sentiments_score = pd.DataFrame.from_dict(scores)
df = df.join(sentiments_score)
df

ValueError: columns overlap but no suffix specified: Index(['Compound', 'Positive', 'Negative', 'Neutral'], dtype='object')

In [37]:
dfnew= df[['Date','Compound', 'Positive', 'Negative', 'Neutral']]

In [42]:
dfnew[''Compound', 'Positive', 'Negative', 'Neutral''].groupby(pd.Grouper(key='Date', freq='1440Min', convention='start')).size()

SyntaxError: invalid syntax (<ipython-input-42-1adcae695bd5>, line 1)

In [50]:
dfnew['Date'] = dfnew['Date'].apply(lambda x: pd.Timestamp(x).strftime('%Y-%m-%d'))
dfnew

,Date,Compound,Positive,Negative,Neutral,date
0,2021-01-01,0.0000,0.000,0.000,1.000,2021-01-01
1,2021-01-01,0.6249,0.369,0.000,0.631,2021-01-01
2,2021-01-01,0.5719,0.252,0.000,0.748,2021-01-01
3,2021-01-01,0.0000,0.000,0.000,1.000,2021-01-01
4,2021-01-01,0.0000,0.000,0.000,1.000,2021-01-01
...,...,...,...,...,...,...
8202,2021-05-31,0.5719,0.190,0.000,0.810,2021-05-31
8203,2021-05-31,0.6597,0.435,0.000,0.565,2021-05-31
8204,2021-05-31,0.2263,0.136,0.095,0.769,2021-05-31
8205,2021-05-31,0.2263,0.095,0.000,0.905,2021-05-31


In [60]:
newdata = dfnew.groupby("Date")['Compound', 'Positive', 'Negative', 'Neutral'].mean()

In [58]:

#df.to_csv(r'C:\Users\Ruchil\OneDrive\MS_Analytics_HU\5thSemester\dataproject\SentimentScore.csv')

In [61]:
newdata.to_csv(r'C:\Users\Ruchil\OneDrive\MS_Analytics_HU\5thSemester\dataproject\SentimentScoreCompound.csv')